In [15]:
import pandas as pd
import numpy as np
from datetime import datetime

cur_state_file = "../second_collection_slower/current_state.csv"
ref_state_file = "../second_collection_slower/reference_state.csv"

def read_csv(cur_state_file, ref_state_file):
    
    cur_states = pd.read_csv(cur_state_file)
    cur_states['timestamp'] = cur_states['time'].apply(lambda t: datetime.strptime(t[:-3], "%Y/%m/%d %H:%M:%S.%f"))
    cur_states['ref_state'] = np.nan
    cur_states = cur_states.sort_values(by='timestamp')
    cur_states.reset_index(drop=True, inplace=True)
    cur_states['state_vector'] = cur_states['state_vector'].apply(lambda s: eval(s)[3:5] + [eval(s)[8]])
    
    ref_states = pd.read_csv(ref_state_file)
    ref_states['timestamp'] = ref_states['time'].apply(lambda t: datetime.strptime(t[:-3], "%Y/%m/%d %H:%M:%S.%f"))
    
    return cur_states, ref_states

cur_states, ref_states = read_csv(cur_state_file, ref_state_file)

In [16]:
def match_cur_ref(cur_states, ref_states):

    ref_index = 0
    ref_timestamp = ref_states['timestamp'][0]
    refs = []
    ref_size = len(ref_states)

    for cur_index, row in cur_states.iterrows():
        if ref_index + 1>= ref_size:
            refs.append(ref_states[['vn', 've', 'yaw']].iloc[ref_index].tolist())
        elif row['timestamp'] > ref_states['timestamp'][ref_index+1]:
            ref_index += 1
            refs.append(ref_states[['vn', 've', 'yaw']].iloc[ref_index].tolist())
        elif row['timestamp'] > ref_states['timestamp'][ref_index]:
            refs.append(ref_states[['vn', 've', 'yaw']].iloc[ref_index].tolist())
        else:
            # Error msg
            print(cur_index, row['timestamp'], ref_states['timestamp'][ref_index])
            refs.append(np.nan)
    cur_states['ref_state'] = refs
    return
match_cur_ref(cur_states, ref_states)

0 2022-04-18 11:33:36.246314 2022-04-18 11:33:36.612339
1 2022-04-18 11:33:36.251873 2022-04-18 11:33:36.612339
2 2022-04-18 11:33:36.257399 2022-04-18 11:33:36.612339
3 2022-04-18 11:33:36.262923 2022-04-18 11:33:36.612339
4 2022-04-18 11:33:36.269033 2022-04-18 11:33:36.612339
5 2022-04-18 11:33:36.274107 2022-04-18 11:33:36.612339
6 2022-04-18 11:33:36.279646 2022-04-18 11:33:36.612339
7 2022-04-18 11:33:36.285180 2022-04-18 11:33:36.612339
8 2022-04-18 11:33:36.290723 2022-04-18 11:33:36.612339
9 2022-04-18 11:33:36.296344 2022-04-18 11:33:36.612339
10 2022-04-18 11:33:36.301875 2022-04-18 11:33:36.612339
11 2022-04-18 11:33:36.307445 2022-04-18 11:33:36.612339
12 2022-04-18 11:33:36.312962 2022-04-18 11:33:36.612339
13 2022-04-18 11:33:36.318499 2022-04-18 11:33:36.612339
14 2022-04-18 11:33:36.324064 2022-04-18 11:33:36.612339
15 2022-04-18 11:33:36.329784 2022-04-18 11:33:36.612339
16 2022-04-18 11:33:36.335245 2022-04-18 11:33:36.612339
17 2022-04-18 11:33:36.340761 2022-04-18 

In [18]:
cur_states.to_csv("processed_states.csv")

In [22]:
cur_states_dropna = cur_states.dropna()
cur_states_list = cur_states_dropna["state_vector"].tolist()
ref_states_list = cur_states_dropna["ref_state"].tolist()

with open("cur_states.npy", "wb") as f:
    np.save(f, np.array(cur_states_list))
with open("ref_states.npy", "wb") as f:
    np.save(f, np.array(ref_states_list))